In [8]:
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import xgboost as xgb
import pandas as pd
import numpy as np

In [9]:
data = pd.read_excel(r'C:\Users\user\Desktop\SKN_AFTER_STUDY\data\33증강데이터_48개.xlsx')
data.head()

,generator_context,category1,category2,input_context,original_index,augmentation_index
0,갑자기 내 책상 위에 놓인 따뜻한 손편지에 마음이 뭉클해졌다.,기쁨,감동,설탕 스틱 껴준거 센스 백점 만점에 천점,20.0,NaN
1,비가 오는데도 친구가 내 좋아하는 카페까지 우산 들고 따라와줘서 마음이 따뜻해졌어.,기쁨,감동,아쓰 산차이 기분 안 좋은 거 알아채고 산차이가 가고 싶다던 토끼집 데려온 거 감동,79.0,NaN
2,"햇살 아래 반짝이는 아이의 눈동자가 마치 작은 보석처럼 빛났다. 그 순간, 세상 모...",기쁨,감동,신데렐라 드레스는 다시 봐도 너무 아름다워. 사람에게 꿈의 물결을 입히다니요.,104.0,NaN
3,이번 전시회 준비하면서 철저하게 세부까지 챙겨준 덕분에 모든 게 완벽하게 마무리돼서...,기쁨,감동,와 민희진 씨 애들 숙소 스타일링까지 맡기면서 신경써 준 거 진짜 좀 대단하네,107.0,NaN
4,비 오는 날 낯선 사람이 내게 담요를 건네며 추위 걱정해 줬다. 마음이 따뜻해져서 ...,기쁨,감동,개감동인 거 자기가 쓰고 있던 우산 나 주고\n자기가 비 맞아가면서 뒤집어준 거야\...,137.0,NaN


In [10]:
def embeddings_model():
  """
  임베딩 모델 초기화
  """
  model = SentenceTransformer("dragonkue/snowflake-arctic-embed-l-v2.0-ko") 
  vec_dim = len(model.encode("dummy_text"))
  print(f"모델 차원: {vec_dim}")
  return model

In [11]:
embeddings_model = embeddings_model()

모델 차원: 1024


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3360 entries, 0 to 3359
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   generator_context   3360 non-null   object 
 1   category1           3360 non-null   object 
 2   category2           3360 non-null   object 
 3   input_context       3360 non-null   object 
 4   original_index      664 non-null    float64
 5   augmentation_index  2696 non-null   float64
dtypes: float64(2), object(4)
memory usage: 157.6+ KB


In [13]:
data['vector'] = ''
data['vector'] = data['generator_context'].apply(lambda x: embeddings_model.encode(x).tolist())

### category1

In [14]:
# 문제 해결: 벡터 컬럼이 이미 리스트 형태로 잘 되어 있음
print("데이터 상태 확인:")
print(f"데이터 개수: {len(data)}")
print(f"벡터 타입: {type(data['vector'].iloc[0])}")
print(f"벡터 길이: {len(data['vector'].iloc[0])}")

# 벡터가 이미 리스트 형태라면 직접 numpy array로 변환
if isinstance(data['vector'].iloc[0], list):
    print("벡터가 리스트 형태입니다. 직접 변환합니다...")
    X = np.vstack(data['vector'].values)
    y = data['category1'].values
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    print("✅ 성공적으로 변환되었습니다!")
else:
    print("벡터 형태에 문제가 있습니다.")

데이터 상태 확인:
데이터 개수: 3360
벡터 타입: <class 'list'>
벡터 길이: 1024
벡터가 리스트 형태입니다. 직접 변환합니다...
X shape: (3360, 1024)
y shape: (3360,)
✅ 성공적으로 변환되었습니다!


In [16]:
# 9. 실제 test_data로 모델 성능 평가

print("📁 실제 test_data 로드 및 평가\n")

# test_data 로드
test_data = pd.read_excel(r'C:\Users\user\Desktop\SKN_AFTER_STUDY\data\증강할데이터33.xlsx')
print("테스트 데이터 기본 정보:")
print(f"데이터 크기: {test_data.shape}")
print(f"컬럼들: {list(test_data.columns)}")
print("\n데이터 샘플:")
print(test_data.head())

# test_data에서 텍스트와 category1 컬럼 확인
print(f"\ntest_data 컬럼 확인:")
for col in test_data.columns:
    print(f"- {col}: {test_data[col].dtype}")

# 텍스트 컬럼과 category1 컬럼 식별 (컬럼명에 따라 조정 필요)
text_column = None
category1_column = None

# 가능한 텍스트 컬럼명들
possible_text_columns = ['context', 'text', 'content', 'sentence', '내용', '문장']
for col in test_data.columns:
    if any(keyword in col.lower() for keyword in possible_text_columns):
        text_column = col
        break

# 가능한 category1 컬럼명들
possible_cat1_columns = ['category1', 'cat1', 'label', '감정', '카테고리1']
for col in test_data.columns:
    if any(keyword in col.lower() for keyword in possible_cat1_columns):
        category1_column = col
        break

print(f"\n식별된 컬럼:")
print(f"텍스트 컬럼: {text_column}")
print(f"Category1 컬럼: {category1_column}")

if text_column and category1_column:
    print(f"\n✅ 필요한 컬럼들을 찾았습니다!")
    print(f"테스트 데이터 개수: {len(test_data)}")
    print(f"Category1 클래스들: {test_data[category1_column].unique()}")
else:
    print(f"\n❌ 필요한 컬럼을 찾을 수 없습니다. 수동으로 지정해주세요.")
    print("사용 가능한 컬럼들:")
    for i, col in enumerate(test_data.columns):
        print(f"{i}: {col}")

📁 실제 test_data 로드 및 평가

테스트 데이터 기본 정보:
데이터 크기: (664, 5)
컬럼들: ['index', 'context', 'annotations_split', 'category1', 'category2']

데이터 샘플:
   index                                            context  \
0      0  보는동안 너무 행복했고 초콜렛이 너무 먹고싶었고 티모시가 잘생겼고 울어!!하는부분이...   
1      1  어릴 때 가 보고 빕스는 거의 처음인데(기억에 없음) 지금 딸기축제 기간이라 만족스...   
2      2  미리 계좌로 환전해둔 돈을 해외에서 환전수수료 없이 인출 가능한 트레블로그라는 카드...   
3      3  요즘 번아웃도 자꾸 올라오고 무기력해서 종강하고 교류하기도 버거운 상태가 와부렀으요ㅠㅠ    
4      4  크라임씬 장똥민이 범행 도구 찾으려고 화장실 탱크 뒤지는데 거기에 진짜 똥 넣어놓은...   

                                   annotations_split category1 category2  
0  [['기쁨', '만족감'], ['기쁨', '만족감'], ['기쁨', '감동'], [...        기쁨       만족감  
1  [['기쁨', '만족감'], ['기쁨', '만족감'], ['기쁨', '만족감'], ...        기쁨       만족감  
2  [['기쁨', '만족감'], ['기쁨', '만족감'], ['기쁨', '만족감'], ...        기쁨       만족감  
3  [['슬픔', '무기력'], ['싫어함(상태)', '무기력'], ['슬픔', '무기...        슬픔       무기력  
4  [['기쁨', '즐거움'], ['기쁨', '통쾌함'], ['기쁨', '통쾌함'], ...        기쁨       즐거움  

test_data 컬럼 확인:
- index: int64


In [17]:
# K-Fold로 평가한 Category1 모델로 test_data 예측 및 classification_report

print("🎯 Category1 모델의 test_data 성능 평가\n")

# 1. 학습 데이터 X, y 변수 정의 (필요시)
if 'X' not in locals():
    print("X 변수를 재정의합니다...")
    X = np.vstack(data['vector'].values)
    y = data['category1'].values
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")

# 2. y_encoded 정의 (필요시)
if 'y_encoded' not in locals():
    print("y_encoded 변수를 재정의합니다...")
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    print(f"y_encoded shape: {y_encoded.shape}")

# 3. test_data의 텍스트를 벡터로 변환
print("📝 test_data 텍스트 임베딩 중...")
test_texts = test_data['context'].fillna('').astype(str).tolist()
test_vectors = []

for text in test_texts:
    vector = embeddings_model.encode(text)
    test_vectors.append(vector)

test_X = np.vstack(test_vectors)
test_y_actual = test_data['category1'].values

print(f"✅ 임베딩 완료: {test_X.shape}")
print(f"실제 라벨: {len(test_y_actual)}")

# 4. 전체 학습 데이터로 최종 모델 학습
print("\n🔄 전체 학습 데이터로 최종 모델 학습...")
final_cat1_model = xgb.XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    tree_method="hist",
    n_jobs=-1
)

# 전체 학습 데이터로 모델 학습
final_cat1_model.fit(X, y_encoded)
print("✅ 최종 모델 학습 완료!")

# 5. test_data로 예측 수행
print("\n🎯 test_data 예측 수행...")
test_y_pred_encoded = final_cat1_model.predict(test_X)
test_y_pred = le.inverse_transform(test_y_pred_encoded)

# 6. 학습 클래스와 테스트 클래스 비교
train_classes = set(le.classes_)
test_actual_classes = set(test_y_actual)
test_pred_classes = set(test_y_pred)

print(f"\n📋 클래스 정보:")
print(f"학습 클래스 수: {len(train_classes)}")
print(f"학습 클래스: {sorted(train_classes)}")
print(f"테스트 실제 클래스 수: {len(test_actual_classes)}")  
print(f"테스트 실제 클래스: {sorted(test_actual_classes)}")

# 학습에 없는 클래스 확인
unseen_classes = test_actual_classes - train_classes
if unseen_classes:
    print(f"⚠️ 학습에 없던 클래스들: {unseen_classes}")
    
common_classes = train_classes & test_actual_classes
print(f"공통 클래스 수: {len(common_classes)}")

# 7. classification_report 생성
print(f"\n📊 Classification Report:")
print("=" * 80)

if len(unseen_classes) == 0:
    # 모든 클래스가 공통인 경우
    test_y_actual_encoded = le.transform(test_y_actual)
    report = classification_report(
        test_y_actual_encoded, 
        test_y_pred_encoded, 
        target_names=le.classes_
    )
    print(report)
    
    # 전체 정확도
    accuracy = (test_y_pred == test_y_actual).mean()
    print(f"\n🎯 전체 정확도: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"평가 데이터: {len(test_y_actual)}개 모두 평가")
    
else:
    # 공통 클래스만 필터링해서 평가
    mask = np.array([actual in common_classes for actual in test_y_actual])
    filtered_actual = test_y_actual[mask]
    filtered_pred = test_y_pred[mask]
    
    filtered_actual_encoded = le.transform(filtered_actual)
    filtered_pred_encoded = le.transform(filtered_pred)
    
    # 공통 클래스에 대한 라벨과 인덱스 매핑
    common_class_labels = [cls for cls in le.classes_ if cls in common_classes]
    common_class_indices = [le.transform([cls])[0] for cls in common_class_labels]
    
    report = classification_report(
        filtered_actual_encoded,
        filtered_pred_encoded,
        target_names=common_class_labels,
        labels=common_class_indices
    )
    print(report)
    
    # 필터링된 데이터의 정확도
    accuracy = (filtered_pred == filtered_actual).mean()
    print(f"\n🎯 정확도 (공통 클래스만): {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"평가 데이터: {len(filtered_actual)}/{len(test_y_actual)}개")

# 8. 예측 샘플 출력
print(f"\n🔍 예측 샘플 (처음 10개):")
print("-" * 90)

for i in range(min(10, len(test_texts))):
    text = test_texts[i][:60] + "..." if len(test_texts[i]) > 60 else test_texts[i]
    actual = test_y_actual[i]
    predicted = test_y_pred[i]
    status = "✅" if actual == predicted else "❌"
    
    print(f"{i+1:2d}. {status} 실제: {actual:<12} 예측: {predicted:<12}")
    print(f"    텍스트: {text}")
    print()

# 9. 클래스별 성능 요약
print(f"\n📈 클래스별 성능 요약:")
print("-" * 60)

from collections import defaultdict
class_stats = defaultdict(lambda: {'total': 0, 'correct': 0})

for actual, pred in zip(test_y_actual, test_y_pred):
    if actual in common_classes:  # 공통 클래스만 계산
        class_stats[actual]['total'] += 1
        if actual == pred:
            class_stats[actual]['correct'] += 1

print(f"{'클래스':<15} {'전체':<8} {'정답':<8} {'정확도':<10}")
print("-" * 50)

for class_name, stats in sorted(class_stats.items()):
    if stats['total'] > 0:
        class_accuracy = stats['correct'] / stats['total']
        print(f"{class_name:<15} {stats['total']:<8} {stats['correct']:<8} {class_accuracy:.4f}")

🎯 Category1 모델의 test_data 성능 평가

y_encoded 변수를 재정의합니다...
y_encoded shape: (3360,)
📝 test_data 텍스트 임베딩 중...
✅ 임베딩 완료: (664, 1024)
실제 라벨: 664

🔄 전체 학습 데이터로 최종 모델 학습...
✅ 최종 모델 학습 완료!

🎯 test_data 예측 수행...

📋 클래스 정보:
학습 클래스 수: 10
학습 클래스: ['기쁨', '두려움', '미움(상대방)', '분노', '사랑', '수치심', '슬픔', '싫어함(상태)', '욕망', '중립']
테스트 실제 클래스 수: 10
테스트 실제 클래스: ['기쁨', '두려움', '미움(상대방)', '분노', '사랑', '수치심', '슬픔', '싫어함(상태)', '욕망', '중립']
공통 클래스 수: 10

📊 Classification Report:
              precision    recall  f1-score   support

          기쁨       0.71      0.70      0.70       188
         두려움       0.89      0.11      0.20        72
     미움(상대방)       0.33      0.72      0.46        43
          분노       0.29      0.42      0.34        36
          사랑       0.60      0.13      0.21        47
         수치심       0.25      0.04      0.07        25
          슬픔       0.64      0.44      0.53       117
     싫어함(상태)       0.19      0.10      0.13        49
          욕망       0.29      0.72      0.42        82
          

c:\Users\user\anaconda3\envs\skn_after_study\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\user\anaconda3\envs\skn_after_study\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\user\anaconda3\envs\skn_after_study\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capi

In [18]:
# Category2 모델로 test_data 예측 및 classification_report

print("\n" + "="*80)
print("🎯 Category2 모델의 test_data 성능 평가")
print("="*80)

# 1. 필요한 변수들 정의 (필요시)
if 'X_combined' not in locals():
    print("Category2 학습용 변수들을 재정의합니다...")
    
    # OneHotEncoder for Category1
    from sklearn.preprocessing import OneHotEncoder
    cat1_encoder = OneHotEncoder(sparse_output=False)
    y_cat1_onehot = cat1_encoder.fit_transform(y.reshape(-1, 1))
    
    # Combined features for Category2
    X_combined = np.hstack([X, y_cat1_onehot])
    
    # Category2 encoder
    y_cat2 = data['category2'].values
    le_cat2 = LabelEncoder()
    y_cat2_encoded = le_cat2.fit_transform(y_cat2)
    
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    print(f"y_cat1_onehot shape: {y_cat1_onehot.shape}")
    print(f"X_combined shape: {X_combined.shape}")
    print(f"y_cat2 shape: {y_cat2.shape}")
    print(f"y_cat2_encoded shape: {y_cat2_encoded.shape}")
    
    # 크기 확인 및 수정
    if X_combined.shape[0] != y_cat2_encoded.shape[0]:
        print(f"⚠️ 크기 불일치 감지: X_combined {X_combined.shape[0]} vs y_cat2_encoded {y_cat2_encoded.shape[0]}")
        min_size = min(X_combined.shape[0], y_cat2_encoded.shape[0])
        X_combined = X_combined[:min_size]
        y_cat2_encoded = y_cat2_encoded[:min_size]
        print(f"✅ 크기 조정 완료: {X_combined.shape[0]} rows")

# 2. Category1을 먼저 예측해야 Category2를 예측할 수 있음
print("\n📝 Category2 예측을 위한 데이터 준비...")

# test_data의 실제 category1과 category2
test_y_actual_cat1 = test_data['category1'].values
test_y_actual_cat2 = test_data['category2'].values

print(f"테스트 데이터:")
print(f"- Category1 실제값: {len(test_y_actual_cat1)}개")
print(f"- Category2 실제값: {len(test_y_actual_cat2)}개")

# Category1 예측값을 사용하여 Category2 예측용 특성 생성
print("\n🔧 Category1 예측값으로 Category2 예측용 특성 생성...")

# Category1 예측값을 원핫인코딩
test_cat1_onehot = cat1_encoder.transform(test_y_pred.reshape(-1, 1))
test_X_combined = np.hstack([test_X, test_cat1_onehot])

print(f"test_X shape: {test_X.shape}")
print(f"test_cat1_onehot shape: {test_cat1_onehot.shape}")
print(f"✅ 결합된 특성: {test_X_combined.shape}")

# 3. 전체 학습 데이터로 Category2 최종 모델 학습
print("\n🔄 전체 학습 데이터로 Category2 최종 모델 학습...")
print(f"학습 데이터 확인: X_combined {X_combined.shape}, y_cat2_encoded {y_cat2_encoded.shape}")

final_cat2_model = xgb.XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    tree_method="hist",
    n_jobs=-1
)

# 전체 학습 데이터로 Category2 모델 학습
final_cat2_model.fit(X_combined, y_cat2_encoded)
print("✅ Category2 최종 모델 학습 완료!")

# 4. test_data로 Category2 예측 수행
print("\n🎯 test_data Category2 예측 수행...")
test_y_pred_cat2_encoded = final_cat2_model.predict(test_X_combined)
test_y_pred_cat2 = le_cat2.inverse_transform(test_y_pred_cat2_encoded)

# 5. 학습 클래스와 테스트 클래스 비교 (Category2)
train_classes_cat2 = set(le_cat2.classes_)
test_actual_classes_cat2 = set(test_y_actual_cat2)

print(f"\n📋 Category2 클래스 정보:")
print(f"학습 클래스 수: {len(train_classes_cat2)}")
print(f"테스트 실제 클래스 수: {len(test_actual_classes_cat2)}")

# 학습에 없는 클래스 확인
unseen_classes_cat2 = test_actual_classes_cat2 - train_classes_cat2
if unseen_classes_cat2:
    print(f"⚠️ 학습에 없던 Category2 클래스들: {unseen_classes_cat2}")

common_classes_cat2 = train_classes_cat2 & test_actual_classes_cat2
print(f"공통 클래스 수: {len(common_classes_cat2)}")

# 6. Category2 Classification Report 생성
print(f"\n📊 Category2 Classification Report:")
print("=" * 80)

if len(unseen_classes_cat2) == 0:
    # 모든 클래스가 공통인 경우
    test_y_actual_cat2_encoded = le_cat2.transform(test_y_actual_cat2)
    report = classification_report(
        test_y_actual_cat2_encoded,
        test_y_pred_cat2_encoded,
        target_names=le_cat2.classes_
    )
    print(report)
    
    # 전체 정확도
    accuracy_cat2 = (test_y_pred_cat2 == test_y_actual_cat2).mean()
    print(f"\n🎯 Category2 전체 정확도: {accuracy_cat2:.4f} ({accuracy_cat2*100:.2f}%)")
    print(f"평가 데이터: {len(test_y_actual_cat2)}개 모두 평가")
    
else:
    # 공통 클래스만 필터링해서 평가
    mask_cat2 = np.array([actual in common_classes_cat2 for actual in test_y_actual_cat2])
    filtered_actual_cat2 = test_y_actual_cat2[mask_cat2]
    filtered_pred_cat2 = test_y_pred_cat2[mask_cat2]
    
    filtered_actual_cat2_encoded = le_cat2.transform(filtered_actual_cat2)
    filtered_pred_cat2_encoded = le_cat2.transform(filtered_pred_cat2)
    
    # 공통 클래스에 대한 라벨과 인덱스 매핑
    common_class_labels_cat2 = [cls for cls in le_cat2.classes_ if cls in common_classes_cat2]
    common_class_indices_cat2 = [le_cat2.transform([cls])[0] for cls in common_class_labels_cat2]
    
    report = classification_report(
        filtered_actual_cat2_encoded,
        filtered_pred_cat2_encoded,
        target_names=common_class_labels_cat2,
        labels=common_class_indices_cat2
    )
    print(report)
    
    # 필터링된 데이터의 정확도
    accuracy_cat2 = (filtered_pred_cat2 == filtered_actual_cat2).mean()
    print(f"\n🎯 Category2 정확도 (공통 클래스만): {accuracy_cat2:.4f} ({accuracy_cat2*100:.2f}%)")
    print(f"평가 데이터: {len(filtered_actual_cat2)}/{len(test_y_actual_cat2)}개")

# 7. Category2 예측 샘플 출력
print(f"\n🔍 Category2 예측 샘플 (처음 10개):")
print("-" * 100)

for i in range(min(10, len(test_texts))):
    text = test_texts[i][:50] + "..." if len(test_texts[i]) > 50 else test_texts[i]
    actual_cat1 = test_y_actual_cat1[i]
    pred_cat1 = test_y_pred[i]
    actual_cat2 = test_y_actual_cat2[i]
    pred_cat2 = test_y_pred_cat2[i]
    status_cat2 = "✅" if actual_cat2 == pred_cat2 else "❌"
    
    print(f"{i+1:2d}. {status_cat2} Cat1: {actual_cat1} → {pred_cat1} | Cat2: {actual_cat2:<12} → {pred_cat2:<12}")
    print(f"    텍스트: {text}")
    print()

# 8. Category2 클래스별 성능 요약
print(f"\n📈 Category2 클래스별 성능 요약:")
print("-" * 60)

class_stats_cat2 = defaultdict(lambda: {'total': 0, 'correct': 0})

for actual, pred in zip(test_y_actual_cat2, test_y_pred_cat2):
    if actual in common_classes_cat2:  # 공통 클래스만 계산
        class_stats_cat2[actual]['total'] += 1
        if actual == pred:
            class_stats_cat2[actual]['correct'] += 1

print(f"{'클래스':<15} {'전체':<8} {'정답':<8} {'정확도':<10}")
print("-" * 50)

for class_name, stats in sorted(class_stats_cat2.items()):
    if stats['total'] > 0:
        class_accuracy = stats['correct'] / stats['total']
        print(f"{class_name:<15} {stats['total']:<8} {stats['correct']:<8} {class_accuracy:.4f}")

# 9. Category1 vs Category2 성능 비교
print(f"\n📊 최종 성능 비교:")
print("=" * 60)
print(f"Category1 정확도: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Category2 정확도: {accuracy_cat2:.4f} ({accuracy_cat2*100:.2f}%)")

if accuracy > accuracy_cat2:
    print("✅ Category1 분류가 더 정확합니다.")
else:
    print("✅ Category2 분류가 더 정확합니다.")


🎯 Category2 모델의 test_data 성능 평가
Category2 학습용 변수들을 재정의합니다...
X shape: (3360, 1024)
y shape: (3360,)
y_cat1_onehot shape: (3360, 10)
X_combined shape: (3360, 1034)
y_cat2 shape: (3360,)
y_cat2_encoded shape: (3360,)

📝 Category2 예측을 위한 데이터 준비...
테스트 데이터:
- Category1 실제값: 664개
- Category2 실제값: 664개

🔧 Category1 예측값으로 Category2 예측용 특성 생성...
test_X shape: (664, 1024)
test_cat1_onehot shape: (664, 10)
✅ 결합된 특성: (664, 1034)

🔄 전체 학습 데이터로 Category2 최종 모델 학습...
학습 데이터 확인: X_combined (3360, 1034), y_cat2_encoded (3360,)
✅ Category2 최종 모델 학습 완료!

🎯 test_data Category2 예측 수행...

📋 Category2 클래스 정보:
학습 클래스 수: 64
테스트 실제 클래스 수: 64
공통 클래스 수: 64

📊 Category2 Classification Report:
              precision    recall  f1-score   support

          갈등       0.00      0.00      0.00         1
          감동       0.33      0.20      0.25        30
          걱정       0.29      0.09      0.13        23
          경멸       0.22      0.33      0.27        18
         고마움       0.54      0.68      0.60        19


c:\Users\user\anaconda3\envs\skn_after_study\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\user\anaconda3\envs\skn_after_study\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\user\anaconda3\envs\skn_after_study\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capi

In [30]:
for context in test_data[test_data['category2']=='불만']['context']:
  print(context)

얘들아 딸기축제인가 딸기장례식인가는 뭐 가지마라
어우 별로를 넘어서 아깝다 그냥
핸드폰 바꿨는데.. 어찌나 전뇌이식이 잘되는지 이전 폰에서 듣던 음악 멈춰둔 부분까지 살려놔서 새로 산 기분이 전혀 안남..
친구가 만들어줬어 어이없어서 받자마자 오열함
아니 맞긴한데
솔직히 주말 껴서 4일…연휴라기엔 너무 눈속임임…사실상 이틀 쉰 거잖아
소인 편의점에서 매일우유 크림빵을 만원 어치 사려고 갔는데 3개를 사기에는 부족한 돈이라 슬펐소이다. 물가가 너무 비싼것 같소.
연휴 이틀이 다 주말에 겹쳐져 있었는데 왜 대체 공휴일은 하루만 주는거야. 부족해, 주말 상관없이 설 연휴 3일 다 완벽하게 보장해 줘.
이렇게 자극적인 드라마 아이들도 다 접할텐데 이젠 수위조절도 안하고 막찍는거같아서 좀 안타까움이 생기네요… ㅠㅠ
이천 햅살 커피프라프치노~! 왠지 고소한 커피맛일꺼란 기대를 엄청안고 주문했는데.. 자그마치 6300원ㅜㅜ 비싼 금액인데 만족스럽진 못했다눈ㅜㅜ


In [20]:
# 10. 통합 예측 파이프라인 구현

class EmotionClassificationPipeline:
    """
    텍스트 입력 → Category1 예측 → Category2 예측 → 종합 평가 파이프라인
    """
    
    def __init__(self, embeddings_model, cat1_model, cat2_model, 
                 cat1_encoder, cat2_encoder, cat1_onehot_encoder):
        self.embeddings_model = embeddings_model
        self.cat1_model = cat1_model
        self.cat2_model = cat2_model
        self.cat1_encoder = cat1_encoder
        self.cat2_encoder = cat2_encoder
        self.cat1_onehot_encoder = cat1_onehot_encoder
        
    def predict_single(self, text):
        """
        단일 텍스트에 대해 카테고리1과 카테고리2를 예측
        
        Args:
            text (str): 예측할 텍스트
        
        Returns:
            dict: 예측 결과 딕셔너리
        """
        # 1. 텍스트 임베딩
        text_vector = self.embeddings_model.encode(text).reshape(1, -1)
        
        # 2. Category1 예측
        cat1_pred_encoded = self.cat1_model.predict(text_vector)[0]
        cat1_pred = self.cat1_encoder.inverse_transform([cat1_pred_encoded])[0]
        cat1_prob = self.cat1_model.predict_proba(text_vector)[0].max()
        
        # 3. Category1 예측값을 사용하여 Category2 예측용 특성 생성
        cat1_onehot = self.cat1_onehot_encoder.transform([[cat1_pred]])
        combined_features = np.hstack([text_vector, cat1_onehot])
        
        # 4. Category2 예측
        cat2_pred_encoded = self.cat2_model.predict(combined_features)[0]
        cat2_pred = self.cat2_encoder.inverse_transform([cat2_pred_encoded])[0]
        cat2_prob = self.cat2_model.predict_proba(combined_features)[0].max()
        
        return {
            'text': text,
            'category1_predicted': cat1_pred,
            'category1_confidence': cat1_prob,
            'category2_predicted': cat2_pred,
            'category2_confidence': cat2_prob
        }
    
    def predict_batch(self, texts):
        """
        여러 텍스트에 대해 배치 예측
        
        Args:
            texts (list): 예측할 텍스트 리스트
        
        Returns:
            list: 예측 결과 리스트
        """
        results = []
        for text in texts:
            result = self.predict_single(text)
            results.append(result)
        return results
    
    def evaluate_with_ground_truth(self, texts, true_cat1, true_cat2):
        """
        실제 정답과 비교하여 성능 평가
        두 카테고리가 모두 맞은 경우만 정답으로 처리
        
        Args:
            texts (list): 예측할 텍스트 리스트
            true_cat1 (list): 실제 category1 라벨
            true_cat2 (list): 실제 category2 라벨
        
        Returns:
            dict: 평가 결과
        """
        predictions = self.predict_batch(texts)
        
        total_count = len(texts)
        cat1_correct = 0
        cat2_correct = 0
        both_correct = 0
        
        detailed_results = []
        
        for i, (pred, actual_cat1, actual_cat2) in enumerate(zip(predictions, true_cat1, true_cat2)):
            cat1_match = pred['category1_predicted'] == actual_cat1
            cat2_match = pred['category2_predicted'] == actual_cat2
            both_match = cat1_match and cat2_match
            
            if cat1_match:
                cat1_correct += 1
            if cat2_match:
                cat2_correct += 1
            if both_match:
                both_correct += 1
            
            detailed_results.append({
                'index': i,
                'text': pred['text'],
                'actual_cat1': actual_cat1,
                'predicted_cat1': pred['category1_predicted'],
                'cat1_match': cat1_match,
                'cat1_confidence': pred['category1_confidence'],
                'actual_cat2': actual_cat2,
                'predicted_cat2': pred['category2_predicted'],
                'cat2_match': cat2_match,
                'cat2_confidence': pred['category2_confidence'],
                'both_correct': both_match
            })
        
        return {
            'total_samples': total_count,
            'category1_accuracy': cat1_correct / total_count,
            'category2_accuracy': cat2_correct / total_count,
            'both_correct_accuracy': both_correct / total_count,  # 핵심 지표
            'category1_correct_count': cat1_correct,
            'category2_correct_count': cat2_correct,
            'both_correct_count': both_correct,
            'detailed_results': detailed_results
        }

# 파이프라인 객체 생성
print("🚀 감정 분류 파이프라인 초기화...")
pipeline = EmotionClassificationPipeline(
    embeddings_model=embeddings_model,
    cat1_model=final_cat1_model,
    cat2_model=final_cat2_model,
    cat1_encoder=le,
    cat2_encoder=le_cat2,
    cat1_onehot_encoder=cat1_encoder
)
print("✅ 파이프라인 초기화 완료!")

🚀 감정 분류 파이프라인 초기화...
✅ 파이프라인 초기화 완료!


In [21]:
# 11. 파이프라인으로 test_data 평가 (두 카테고리 모두 맞은 경우만 정답 처리)

print("🎯 파이프라인으로 test_data 종합 평가")
print("="*80)

# test_data 준비
test_texts = test_data['context'].fillna('').astype(str).tolist()
test_true_cat1 = test_data['category1'].values
test_true_cat2 = test_data['category2'].values

print(f"평가 데이터: {len(test_texts)}개")
print(f"Category1 클래스 수: {len(np.unique(test_true_cat1))}")
print(f"Category2 클래스 수: {len(np.unique(test_true_cat2))}")

# 파이프라인으로 종합 평가 실행
print(f"\n🔄 파이프라인 평가 실행 중...")
evaluation_results = pipeline.evaluate_with_ground_truth(
    test_texts, test_true_cat1, test_true_cat2
)

# 결과 출력
print(f"\n📊 파이프라인 종합 평가 결과:")
print("="*60)
print(f"전체 테스트 샘플 수: {evaluation_results['total_samples']}")
print(f"")
print(f"📈 개별 정확도:")
print(f"  Category1 정확도: {evaluation_results['category1_accuracy']:.4f} ({evaluation_results['category1_correct_count']}/{evaluation_results['total_samples']})")
print(f"  Category2 정확도: {evaluation_results['category2_accuracy']:.4f} ({evaluation_results['category2_correct_count']}/{evaluation_results['total_samples']})")
print(f"")
print(f"🎯 핵심 지표 - 두 카테고리 모두 정답:")
print(f"  종합 정확도: {evaluation_results['both_correct_accuracy']:.4f} ({evaluation_results['both_correct_count']}/{evaluation_results['total_samples']})")
print(f"  종합 정확도: {evaluation_results['both_correct_accuracy']*100:.2f}%")

# 상세 분석
print(f"\n🔍 상세 분석:")
print("-"*60)

# 카테고리별 매치 패턴 분석
both_correct = sum(1 for r in evaluation_results['detailed_results'] if r['both_correct'])
only_cat1_correct = sum(1 for r in evaluation_results['detailed_results'] if r['cat1_match'] and not r['cat2_match'])
only_cat2_correct = sum(1 for r in evaluation_results['detailed_results'] if not r['cat1_match'] and r['cat2_match'])
both_wrong = sum(1 for r in evaluation_results['detailed_results'] if not r['cat1_match'] and not r['cat2_match'])

print(f"두 카테고리 모두 정답: {both_correct}개 ({both_correct/evaluation_results['total_samples']*100:.2f}%)")
print(f"Category1만 정답: {only_cat1_correct}개 ({only_cat1_correct/evaluation_results['total_samples']*100:.2f}%)")
print(f"Category2만 정답: {only_cat2_correct}개 ({only_cat2_correct/evaluation_results['total_samples']*100:.2f}%)")
print(f"둘 다 틀림: {both_wrong}개 ({both_wrong/evaluation_results['total_samples']*100:.2f}%)")

# 샘플 출력 - 두 카테고리 모두 맞은 케이스
print(f"\n✅ 두 카테고리 모두 정답인 샘플들 (처음 10개):")
print("="*100)

correct_samples = [r for r in evaluation_results['detailed_results'] if r['both_correct']]
for i, result in enumerate(correct_samples[:10]):
    text = result['text'][:60] + "..." if len(result['text']) > 60 else result['text']
    print(f"{i+1:2d}. Cat1: {result['actual_cat1']} ✓ | Cat2: {result['actual_cat2']} ✓")
    print(f"    신뢰도: Cat1={result['cat1_confidence']:.3f}, Cat2={result['cat2_confidence']:.3f}")
    print(f"    텍스트: {text}")
    print()

# 샘플 출력 - 둘 다 틀린 케이스  
print(f"\n❌ 두 카테고리 모두 틀린 샘플들 (처음 10개):")
print("="*100)

wrong_samples = [r for r in evaluation_results['detailed_results'] if not r['both_correct'] and not r['cat1_match'] and not r['cat2_match']]
for i, result in enumerate(wrong_samples[:10]):
    text = result['text'][:60] + "..." if len(result['text']) > 60 else result['text']
    print(f"{i+1:2d}. Cat1: {result['actual_cat1']} → {result['predicted_cat1']} | Cat2: {result['actual_cat2']} → {result['predicted_cat2']}")
    print(f"    신뢰도: Cat1={result['cat1_confidence']:.3f}, Cat2={result['cat2_confidence']:.3f}")
    print(f"    텍스트: {text}")
    print()

print(f"\n🔥 결론:")
print(f"이 파이프라인에서 두 카테고리가 모두 정확하게 예측된 경우는 전체의 {evaluation_results['both_correct_accuracy']*100:.2f}%입니다.")

🎯 파이프라인으로 test_data 종합 평가
평가 데이터: 664개
Category1 클래스 수: 10
Category2 클래스 수: 64

🔄 파이프라인 평가 실행 중...

📊 파이프라인 종합 평가 결과:
전체 테스트 샘플 수: 664

📈 개별 정확도:
  Category1 정확도: 0.4654 (309/664)
  Category2 정확도: 0.2063 (137/664)

🎯 핵심 지표 - 두 카테고리 모두 정답:
  종합 정확도: 0.1973 (131/664)
  종합 정확도: 19.73%

🔍 상세 분석:
------------------------------------------------------------
두 카테고리 모두 정답: 131개 (19.73%)
Category1만 정답: 178개 (26.81%)
Category2만 정답: 6개 (0.90%)
둘 다 틀림: 349개 (52.56%)

✅ 두 카테고리 모두 정답인 샘플들 (처음 10개):
 1. Cat1: 기쁨 ✓ | Cat2: 만족감 ✓
    신뢰도: Cat1=0.873, Cat2=0.229
    텍스트: 우연히 보게 된 영상인데, 노래가 너무 좋아서 플리에도 추가하고, 카카오톡 프뮤로도 해놨음. 음원도 좋긴 한...

 2. Cat1: 기쁨 ✓ | Cat2: 만족감 ✓
    신뢰도: Cat1=0.926, Cat2=0.100
    텍스트: 초대박 귀여운 강아디도 있음 우리 자리 와서 인사도 해줌 .. 🥹
좌석이 약간 당황스러운 것 말고 다 좋았던...

 3. Cat1: 기쁨 ✓ | Cat2: 만족감 ✓
    신뢰도: Cat1=0.896, Cat2=0.427
    텍스트: 편의점 초콜릿 중 가장 맛있는 초콜릿이라고 생각하오. 입에 넣자마자 기분 좋아지는 맛이오. 

 4. Cat1: 욕망 ✓ | Cat2: 아쉬움 ✓
    신뢰도: Cat1=0.334, Cat2=0.369
    텍스트: 말들 의식해서 바꿨으면 사과 공지 짧게라도 올렸음 좋았을텐데
이렇게 슬쩍 수정한거 조

In [ ]:
# 12. 새로운 텍스트 예측 데모 함수

def demo_pipeline_prediction():
    """
    새로운 텍스트들에 대해 파이프라인 예측 시연
    """
    print("🚀 파이프라인 예측 시연")
    print("="*80)
    
    # 예제 텍스트들
    demo_texts = [
        "친구가 생일 파티를 준비해줘서 너무 감동받았어",
        "시험 결과가 나쁘게 나와서 정말 실망스럽다", 
        "새로운 직장이 확정되어서 설레고 기대된다",
        "누군가 내 뒷담화를 하는 걸 들어서 화가 난다"
    ]
    
    for i, text in enumerate(demo_texts, 1):
        print(f"\n📝 예제 {i}: {text}")
        print("-"*60)
        
        # 파이프라인으로 예측
        result = pipeline.predict_single(text)
        
        print(f"🎯 예측 결과:")
        print(f"  Category1: {result['category1_predicted']} (신뢰도: {result['category1_confidence']:.3f})")
        print(f"  Category2: {result['category2_predicted']} (신뢰도: {result['category2_confidence']:.3f})")

# 시연 실행
demo_pipeline_prediction()

🚀 파이프라인 예측 시연

📝 예제 1: 친구가 생일 파티를 준비해줘서 너무 감동받았어
------------------------------------------------------------
🎯 예측 결과:
  Category1: 기쁨 (신뢰도: 0.996)
  Category2: 고마움 (신뢰도: 0.795)

📝 예제 2: 시험 결과가 나쁘게 나와서 정말 실망스럽다
------------------------------------------------------------
🎯 예측 결과:
  Category1: 슬픔 (신뢰도: 0.315)
  Category2: 실망 (신뢰도: 0.540)

📝 예제 3: 새로운 직장이 확정되어서 설레고 기대된다
------------------------------------------------------------
🎯 예측 결과:
  Category1: 기쁨 (신뢰도: 0.693)
  Category2: 기대감 (신뢰도: 0.969)

📝 예제 4: 누군가 내 뒷담화를 하는 걸 들어서 화가 난다
------------------------------------------------------------
🎯 예측 결과:
  Category1: 미움(상대방) (신뢰도: 0.834)
  Category2: 치사함 (신뢰도: 0.615)


: 